# Evaluation

In [ ]:
import h5py
import ipywidgets as widgets
from functools import partial

from tqdne.conf import Config
from tqdne.metric import PowerSpectralDensity, MeanSquaredError, BinMetric

In [ ]:
max_num_samples = 50
config = Config()
test_dataset_path = config.datasetdir / config.data_upsample_test
test_file = h5py.File(test_dataset_path, mode="r")
test_waveforms = test_file["waveform"][:max_num_samples]
test_features = test_file["features"][:max_num_samples]
test_file.close()

pred_dataset_path = config.datasetdir / config.data_upsample_train # TODO: placeholder for generated dataset
pred_file = h5py.File(pred_dataset_path, mode="r")
pred_waveforms = pred_file["waveform"][:max_num_samples]
pred_file.close()

## Plotting metrics

In [ ]:
metrics = {"Power Spectral Density": partial(PowerSpectralDensity, fs=config.fs), "Mean Squared Error": MeanSquaredError}

# Create a dropdown for selecting the metric
metric_dropdown = widgets.Dropdown(
    options=metrics.keys(),
    description='Metric:',
)

# Create a slider for selecting the channel
channel_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=2,
    step=1,
    description='Channel:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


# Bin plot checkbox
bin_plot_checkbox = widgets.Checkbox(
    value=False,
    description='Plot bins',
    disabled=False,
    indent=True,
)

# Create a slider for selecting the number of bins
num_bins_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Num bins:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Function to update the plot
def update_plot(metric_name, bin_plot, num_bins=10, channel=0):
    metric = metrics[metric_name](channel=channel)
    if bin_plot:
        metric = BinMetric(metric, num_mag_bins=num_bins, num_dist_bins=num_bins)

    # Compute the metric
    metric.reset()
    metric.update(pred={"high_res": pred_waveforms}, target={"high_res": test_waveforms, "cond": test_features})
    metric.plot().show()

# Create interactive plot
widgets.interact(update_plot, metric_name=metric_dropdown, channel=channel_slider, bin_plot=bin_plot_checkbox, num_bins=num_bins_slider)